In [1]:
!pip install opencv-python
!pip install pytesseract


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import pytesseract
from pytesseract import Output
import os


In [3]:
import cv2
import pytesseract
import os
import re

# Ensure Tesseract OCR is properly configured
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def preprocess_frame(frame):
    """
    Preprocess the frame for better OCR accuracy.

    Args:
        frame (numpy.ndarray): The input video frame.

    Returns:
        numpy.ndarray: The preprocessed frame.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    denoised = cv2.medianBlur(gray, 3)  # Remove noise with median blur
    _, thresh = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)  # Binarize
    return thresh

def clean_ocr_text(text):
    """
    Clean the OCR output by removing noise and unwanted characters.

    Args:
        text (str): The raw OCR output.

    Returns:
        str: The cleaned text.
    """
    # Remove short lines and unwanted characters using regex
    cleaned_text = re.sub(r'[^A-Za-z0-9\s,.!?-]', '', text)
    cleaned_text = '\n'.join([line.strip() for line in cleaned_text.splitlines() if len(line.strip()) > 2])
    return cleaned_text

def extract_text_from_video(video_path, output_file, interval=1):
    """
    Extract text blocks from a video and save them to a file.

    Args:
        video_path (str): Path to the input video.
        output_file (str): Path to the output text file.
        interval (int): Time interval (in seconds) between frames to process.
    """
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_interval = interval * fps  # Convert interval to frames
    frame_count = 0

    # Create or overwrite the output file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(f"Extracted text from {os.path.basename(video_path)}\n\n")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Process frame at the specified interval
        if frame_count % frame_interval == 0:
            timestamp = frame_count / fps  # Calculate timestamp in seconds
            preprocessed_frame = preprocess_frame(frame)  # Preprocess the frame

            # Perform OCR on the preprocessed frame
            text = pytesseract.image_to_string(preprocessed_frame, lang='eng', config='--psm 6 --oem 3')
            cleaned_text = clean_ocr_text(text)

            if cleaned_text:  # Only save non-empty results
                with open(output_file, 'a', encoding='utf-8') as f:
                    # f.write(f"Timestamp: {timestamp:.2f} seconds\n")
                    f.write(cleaned_text + "\n\n")
                print(f"Extracted text at {timestamp:.2f} seconds")

        frame_count += 1

    cap.release()
    print(f"\nText extraction complete. Results saved to {output_file}")

# Example usage
if __name__ == "__main__":
    video_path = r"D:\COURSES\CGAS\Dish-Decode\Video_text_recipe_opencv.mp4"  # Use raw string to avoid escape issues
    output_file = "extracted_text.txt"
    interval = 2  # Process a frame every 2 seconds (adjust as needed)

    extract_text_from_video(video_path, output_file, interval)

# import cv2
# import pytesseract
# import os
# import re

# # Ensure Tesseract OCR is properly configured
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# def preprocess_frame(frame):
#     """
#     Preprocess the frame for better OCR accuracy.

#     Args:
#         frame (numpy.ndarray): The input video frame.

#     Returns:
#         numpy.ndarray: The preprocessed frame.
#     """
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
#     denoised = cv2.medianBlur(gray, 3)  # Remove noise with median blur
#     _, thresh = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)  # Binarize
#     return thresh

# def clean_ocr_text(text):
#     """
#     Clean the OCR output by removing noise and unwanted characters.

#     Args:
#         text (str): The raw OCR output.

#     Returns:
#         str: The cleaned text.
#     """
#     # Remove short lines and unwanted characters using regex
#     cleaned_text = re.sub(r'[^A-Za-z0-9\s,.!?-]', '', text)
#     cleaned_text = '\n'.join([line.strip() for line in cleaned_text.splitlines() if len(line.strip()) > 2])
#     return cleaned_text

# def extract_text_from_video(video_path, output_file, interval=1):
#     """
#     Extract text blocks from a video and save them to a file.
#     Save the first frame with detected text as an image.

#     Args:
#         video_path (str): Path to the input video.
#         output_file (str): Path to the output text file.
#         interval (int): Time interval (in seconds) between frames to process.
#     """
#     cap = cv2.VideoCapture(video_path)
#     fps = int(cap.get(cv2.CAP_PROP_FPS))
#     frame_interval = interval * fps  # Convert interval to frames
#     frame_count = 0
#     first_image_saved = False  # Flag to track if the first frame is saved

#     # Create or overwrite the output file
#     with open(output_file, 'w', encoding='utf-8') as f:
#         f.write(f"Extracted text from {os.path.basename(video_path)}\n\n")

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Process frame at the specified interval
#         if frame_count % frame_interval == 0:
#             timestamp = frame_count / fps  # Calculate timestamp in seconds
#             preprocessed_frame = preprocess_frame(frame)  # Preprocess the frame

#             # Perform OCR on the preprocessed frame
#             text = pytesseract.image_to_string(preprocessed_frame, lang='eng', config='--psm 6 --oem 3')
#             cleaned_text = clean_ocr_text(text)

#             if cleaned_text:  # Only save non-empty results
#                 with open(output_file, 'a', encoding='utf-8') as f:
#                     f.write(cleaned_text + "\n\n")
#                 print(f"Extracted text at {timestamp:.2f} seconds")

#                 # Save the first frame with text as an image
#                 if not first_image_saved:
#                     image_filename = "recipe_image.png"
#                     cv2.imwrite(image_filename, frame)
#                     print(f"First frame with text saved as {image_filename}")
#                     first_image_saved = True

#         frame_count += 1

#     cap.release()
#     print(f"\nText extraction complete. Results saved to {output_file}")

# # Example usage
# if __name__ == "__main__":
#     video_path = r"D:\COURSES\CGAS\Dish-Decode\thirdVideo.mp4"  # Use raw string to avoid escape issues
#     output_file = "extracted_text.txt"
#     interval = 1  # Process a frame every 1 second (adjust as needed)

#     extract_text_from_video(video_path, output_file, interval)


Extracted text at 0.00 seconds
Extracted text at 2.00 seconds
Extracted text at 4.00 seconds
Extracted text at 6.00 seconds
Extracted text at 8.00 seconds
Extracted text at 10.00 seconds
Extracted text at 12.00 seconds
Extracted text at 24.00 seconds
Extracted text at 26.00 seconds
Extracted text at 32.00 seconds
Extracted text at 36.00 seconds
Extracted text at 38.00 seconds
Extracted text at 40.00 seconds
Extracted text at 42.00 seconds
Extracted text at 48.00 seconds
Extracted text at 50.00 seconds
Extracted text at 52.00 seconds
Extracted text at 54.00 seconds
Extracted text at 56.00 seconds
Extracted text at 58.00 seconds
Extracted text at 60.00 seconds
Extracted text at 62.00 seconds
Extracted text at 68.00 seconds
Extracted text at 74.00 seconds
Extracted text at 76.00 seconds
Extracted text at 78.00 seconds
Extracted text at 82.00 seconds
Extracted text at 86.00 seconds
Extracted text at 88.00 seconds
Extracted text at 90.00 seconds
Extracted text at 92.00 seconds
Extracted tex